# Rule Generation

In this notebook we will showcase the efficiency of PAM for rule generation.

We will follow the procedure described in Section 3.4 for the applicaiton of rule generation.

The process is simple:
1. First download the **YAGO3-10** dataset. The .tsv with the triples is expected to be in folder in the same directory as this notebook with the name "data".
2. Create the **lossless** $1$-hop PAM  and the **k-hop** PAMs $P^k$ which hold the corresponding paths.
3. Find paths $Path^k$ that may imply a head relation $r_H$ through a simple check: $P[A,B] = r_H$ and $P^k[A,B] = Path^k \neq 0$.


## Initial imports

In [1]:
import tqdm
import time
import string

import pandas as pd
import numpy as np

from itertools import permutations, product
from pam_creation import create_pam_matrices
from utils import load_data, get_primefactors

# Loading the data

We simply read the collection of triples in a dataframe

In [2]:
path_to_files = './data/YAGO3-10'
project = "YAGO3-10"


time_s = time.time()
_, df_train, _, _, _ = load_data(path_to_files, project)

unique_rels = sorted(list(df_train['rel'].unique()))
unique_nodes = sorted(set(df_train['head'].values.tolist() + df_train['tail'].values.tolist()))
print(f'# of unique rels: {len(unique_rels)} \t | # of unique nodes: {len(unique_nodes)}')


time_prev = time.time()
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')

Total: 735915 triples in train + eval!)
In train: 732556
In valid: 3359
In test: 3390
# of unique rels: 36 	 | # of unique nodes: 122780
Total time: 0.91459 secs (0.02 mins)


In [3]:
df_train.head()

,head,rel,tail
0,Chatou,isLocatedIn,France
1,Toni_Kuivasto,isAffiliatedTo,Helsingin_Jalkapalloklubi
2,Albrecht_Dürer,diedIn,Nuremberg
3,Edwin_Holliday,isAffiliatedTo,Hereford_United_F.C.
4,William_Hopper,actedIn,The_Bad_Seed_(1956_film)


## Create the $1$-hop **lossless** PAM and its powers.

Here we limit $k=3$.

In [4]:
max_k = 3
pam_1hop_lossless, pam_powers, node2id, rel2id = create_pam_matrices(df_train, 
                                                                     max_order=max_k, 
                                                                     spacing_strategy='step_100', 
                                                                     use_log=False)

# of unique rels: 36 	 | # of unique nodes: 122780
(122780, 122780) Sparsity: 100.00 %
Sparsity 2-hop: 99.97 %
Sparsity 3-hop: 99.84 %


# Generate possible rules

The procedure is as follows:
1. We first find all pairs of $(i,j)$ for which $P[i,j] = r_H = \neq 0 $.
2. Then we keep track of all the different values for these $(i,j)$,that $P^k[i,j] =  Path^k = \neq 0$.
3. We aggregate all of these rules in a DataFrame, keeping also frequencies and other characteristics of the rules (e.g. support, confidence etc.)

## 1-hop rules

Rules that have only one atom in the body.
E.g. $knows(A,B)->friend(A,B)$

In [5]:
all_rules = []
cur_power = pam_1hop_lossless
rows, cols = pam_1hop_lossless.nonzero()
head_rels = np.array(pam_1hop_lossless[rows,cols]).flatten()
cur_rules = []

time_s = time.time()

# Iterate over all non-zero # P[i,j]
for index_nonzero, body_rel in tqdm.tqdm_notebook(enumerate(head_rels.flatten()), total=len(head_rels)):
    head_rels_cur = get_primefactors(body_rel)
    if len(head_rels_cur) > 1 :
        # Subrelation rules
        for comb in permutations(head_rels_cur, 2):
            if comb[0] in rel2id.values():
                cur_rules.append(
                    {
                        'head_rel':comb[0], 
                        "body":comb[1], 
                        "hop":1, 
                        'type':'same_cell',
                        'row': rows[index_nonzero],
                        'col': cols[index_nonzero]
                     }
                )

# Keep all rules
df_rules = pd.DataFrame(cur_rules)
# Aggregate the patterns by their body value
grouped_by_head_and_body = df_rules.groupby(["head_rel", 'type'])['body'].value_counts().to_dict()
# Calculate the support of the heads
head_support = df_rules['head_rel'].value_counts().to_dict()



all_bodies = []
for value in cur_power.data:
    all_bodies.extend(get_primefactors(value))
# Calculate the support of the bodies
body_support = pd.Series(all_bodies).value_counts().to_dict()

for key in grouped_by_head_and_body:
    head_rel, type_, body_rel  = key
    all_rules.append({
        "head_rel": head_rel, 
        'body':body_rel, 
        'head_body_count':grouped_by_head_and_body[key], 
        'body_count':body_support[body_rel],
        'head_count':head_support[head_rel],
        'hop':1, 
        'type':type_})

print(f'Total # rules: {len(all_rules)}')

time_prev = time.time()
time_needed = time_s - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')


/tmp/ipykernel_2379803/3213398325.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index_nonzero, body_rel in tqdm.tqdm_notebook(enumerate(head_rels.flatten()), total=len(head_rels)):


  0%|          | 0/725199 [00:00<?, ?it/s]

Total # rules: 104
Total time: 0.00000 secs (0.00 mins)


## $k-$hop rules

Rules of the form:
$r1(A,B) \wedge r2(B,C) \wedge ... rk(Y,Z) - > r_H(A,Z)$

In [6]:
max_hop = len(pam_powers)
laplace_smoothing_alpha = 0

time_s = time.time()

rows, cols = pam_1hop_lossless.nonzero()
head_rels = np.array(pam_1hop_lossless[rows,cols]).flatten()
for cur_hop in range(1,max_hop):
    cur_power = pam_powers[cur_hop]
    cur_rules = []
    for index_nonzero, body_rel in enumerate(np.array(cur_power[rows, cols]).flatten()):
        cur_prime_factors = get_primefactors(head_rels[index_nonzero])
        if body_rel != 0:
            for head_rel in cur_prime_factors:
                if head_rel in rel2id.values():
                    cur_rules.append(
                        {
                            'head_rel':head_rel, 
                            "body":body_rel, 
                            "hop":cur_hop + 1, 
                            'type':'same_cell',
                            'row': rows[index_nonzero],
                            'col': cols[index_nonzero]
                        }
                    )
                
    df_rules = pd.DataFrame(cur_rules)
    grouped_by_head_and_body = df_rules.groupby(["head_rel", 'type'])['body'].value_counts().to_dict()
    head_support = df_rules['head_rel'].value_counts().to_dict()
    body_support = pd.Series(cur_power.data).value_counts().to_dict()
    for key in grouped_by_head_and_body:
        head_value, type_, body_value  = key
        all_rules.append({
            "head_rel": head_value, 
            'body':body_value, 
            'head_body_count':grouped_by_head_and_body[key], 
            'body_count':body_support[body_value],
            'head_count':head_support[head_value],
            'hop':cur_hop + 1,
            'type':type_})

    print(f'{cur_hop + 1}-hop level rules: {len(grouped_by_head_and_body)}')
print(f'Total # rules: {len(all_rules)}')

time_prev = time.time()
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')


all_rules_df = pd.DataFrame(all_rules)
all_rules_df['conf'] = all_rules_df['head_body_count'] / (all_rules_df['body_count'] + laplace_smoothing_alpha)
all_rules_df['head_coverage'] = all_rules_df['head_body_count'] / (all_rules_df['head_count'])
all_rules_df['score'] = 2*(all_rules_df['conf']*all_rules_df['head_coverage']/(all_rules_df['head_coverage'] + all_rules_df['conf'] ))

all_rules_df['body_head_str'] = all_rules_df['body'].astype(str) + "_" + all_rules_df['head_rel'].astype(str)
all_rules_df.sort_values(['score'], ascending=False)


2-hop level rules: 4320
3-hop level rules: 24015
Total # rules: 28439
Total time: 1.64373 secs (0.03 mins)


,head_rel,body,head_body_count,body_count,head_count,hop,type,conf,head_coverage,score,body_head_str
71,2851,1049,992,2566,1587,1,same_cell,3.865939e-01,0.625079,4.777269e-01,1049_2851
9902,1049,25500143730021451,619,731,2339,3,same_cell,8.467852e-01,0.264643,4.032573e-01,25500143730021451_1049
1180,2221,2556371,718,2335,1252,2,same_cell,3.074946e-01,0.573482,4.003345e-01,2556371_2221
21132,2953,11743094698,1,1,4,3,same_cell,1.000000e+00,0.250000,4.000000e-01,11743094698_2953
865,1361,1566511,1391,2230,5155,2,same_cell,6.237668e-01,0.269835,3.767095e-01,1566511_1361
...,...,...,...,...,...,...,...,...,...,...,...
7918,839,8068178897,3,3101484,4737,3,same_cell,9.672789e-07,0.000633,1.931608e-06,8068178897_839
15578,2647,33570932693,1,1258733,14,3,same_cell,7.944497e-07,0.071429,1.588882e-06,33570932693_2647
205,211,9616423,1,2403292,1011,2,same_cell,4.160959e-07,0.000989,8.318419e-07,9616423_211
10021,1049,2029065253,1,2618367,2339,3,same_cell,3.819174e-07,0.000428,7.631531e-07,2029065253_1049


We have generated $28439$ patterns that may imply (with different confidence, support levels) different relations in a matter of seconds.

In [7]:

id2rel = dict(zip(rel2id.values(), rel2id.keys()))

def generate_rule_string(chain_of_relations:list[str], head_rel:str, type_of_rule:str):
    """Helper function to create the rule

    Args:
        chain_of_relations (list[str]): The chain of relations followed.
        head_rel (str): The relation implied in the head.
        type_of_rule (str): The type of the rule. 'same_cell' or 'symmetric'.
        The difference in the cases is:
        'same_cell': r1(A,B)^r2(B,C)->r3(A,C)
        or
        'symmetric': r1(A,B)^r2(B,C)->r3(C,A)

    Returns:
        str: the rule as a Horn clause
    """

    possible_letters = string.ascii_uppercase
    first_letter = possible_letters[0]
    rule_str = ""
    for i_rel, relation in enumerate(chain_of_relations):
        next_letter = possible_letters[possible_letters.index(first_letter) + 1]
        rule_str += f"{relation}({first_letter}, {next_letter}) ^ "
        first_letter = next_letter

    rule_str = rule_str[:-2]
    
    if type_of_rule == 'same_cell':
        head = f"-> {head_rel}({possible_letters[0]}, {next_letter})"
    else:
        head = f"-> {head_rel}({next_letter}, {possible_letters[0]})"

    rule_str += head
    return rule_str

In [26]:
num_hops = 2
rules_mapped = []
for i,row in all_rules_df[(all_rules_df['hop']<=num_hops) & (all_rules_df['score']>=0.01)].sort_values(['score'], ascending=False).iterrows():
    try:
        chain = [id2rel[pf] for pf in get_primefactors(row['body'])]
    except KeyError:
        continue
    if len(chain) <= num_hops:
        pass
    
    head_rel = id2rel[row['head_rel']]
    rule_str = generate_rule_string(chain, head_rel, row['type'])
    print(f"{row['score']:.4f} : {rule_str}\t")
    print('Chain:',chain, [pf for pf in get_primefactors(row['body'])])
    print('Predicts:', id2rel[row['head_rel']],[row['head_rel']], f"({row['type']})")
    print()
    rules_mapped.append({'Score': row['score'],'Head Rel':head_rel, 'Rule': rule_str})
rules_df = pd.DataFrame(rules_mapped)

0.4777 : hasCapital(A, B) -> isLocatedIn(A, B)	
Chain: ['hasCapital'] [1049]
Predicts: isLocatedIn [2851] (same_cell)

0.4003 : hasChild(A, B) ^ isAffiliatedTo(B, C) -> isAffiliatedTo(A, C)	
Chain: ['hasChild', 'isAffiliatedTo'] [1151, 2221]
Predicts: isAffiliatedTo [2221] (same_cell)

0.3767 : hasChild(A, B) ^ hasGender(B, C) -> hasGender(A, C)	
Chain: ['hasChild', 'hasGender'] [1151, 1361]
Predicts: hasGender [1361] (same_cell)

0.3077 : directed(A, B) -> created(A, B)	
Chain: ['directed'] [419]
Predicts: created [107] (same_cell)

0.3043 : isLocatedIn(A, B) -> isLocatedIn(A, B)	
Chain: ['isLocatedIn'] [2851]
Predicts: isLocatedIn [2851] (same_cell)

0.3030 : isKnownFor(A, B) -> influences(A, B)	
Chain: ['isKnownFor'] [2647]
Predicts: influences [2113] (same_cell)

0.2999 : worksAt(A, B) -> graduatedFrom(A, B)	
Chain: ['worksAt'] [3593]
Predicts: graduatedFrom [733] (same_cell)

0.2740 : exports(A, B) -> imports(A, B)	
Chain: ['exports'] [631]
Predicts: imports [2011] (same_cell)

0.

## Focus on the worksAt relation

In [42]:
pd.set_option('display.max_colwidth', None)
rules_df[rules_df['Head Rel'] == 'worksAt'].sort_values('Score', ascending=False)


,Score,Head Rel,Rule
28,0.155268,worksAt,"graduatedFrom(A, B) ^ hasAcademicAdvisor(B, C) ^ worksAt(C, D) -> worksAt(A, D)"
30,0.154072,worksAt,"hasAcademicAdvisor(A, B) ^ worksAt(B, C) -> worksAt(A, C)"
31,0.150616,worksAt,"graduatedFrom(A, B) -> worksAt(A, B)"
64,0.079935,worksAt,"graduatedFrom(A, B) ^ hasAcademicAdvisor(B, C) -> worksAt(A, C)"
115,0.040562,worksAt,"influences(A, B) ^ worksAt(B, C) -> worksAt(A, C)"
122,0.033708,worksAt,"graduatedFrom(A, B) ^ influences(B, C) ^ worksAt(C, D) -> worksAt(A, D)"
171,0.019118,worksAt,"graduatedFrom(A, B) ^ influences(B, C) -> worksAt(A, C)"
197,0.012821,worksAt,"graduatedFrom(A, B) ^ hasChild(B, C) ^ worksAt(C, D) -> worksAt(A, D)"
224,0.010025,worksAt,"owns(A, B) ^ worksAt(B, C) -> worksAt(A, C)"


# Conclusion

We show an elegant and fast way of utilizing PAM for rule generation.

The whole procedure takes less than 30 seconds on a commercial laptop and we generate almost 30 thousand rules (for body length up to $k=3$ atoms).

